In [1]:
import numpy as np
import pandas as pd
import gensim
import collections 
import time
import random

# 불용어 추출 

불용어(Stopwords) 는 과하게 빈도수가 높아 자연어 처리에 도움을 주지 않는 문자들로, 이 code는 해당 stopwords를 추출하기 위함이다. 

Stopwords를 성능에 직접적으로 관여하는 요소는 아니다. 
그러나 최소한의 제거도 하지 않으면 Model 성능 저하 우려가 있다. 

In [78]:
from collections import defaultdict
import re 
from eunjeon import Mecab

stop_text = open("KoreanStopwords.txt", "r", encoding = "utf-8")
stop_list = []
for line in stop_text :
    line = line[:-1]
    stop_list.append(line)
stop_text.close()
stop_list.extend(["\xa0","\u200b"])

In [44]:
komoran = Komoran()
mecab = Mecab()

In [4]:
flist = ["processed_SNS_1.xlsx","processed_SNS_2.xlsx","processed_SNS_3.xlsx","processed_SNS_4.xlsx","processed_SNS_5.xlsx", "processed_SNS_6.xlsx", "processed_SNS_7.xlsx", "processed_SNS_8.xlsx"]

In [5]:
def sample_df(fname):
    df = pd.read_excel(fname)
    df_percent = df.sample(frac=0.08) #proportion of sampling 
    return df_percent

In [6]:
#assign first sample dataframe 
collected_df = sample_df(flist[0]) 

In [15]:
for i in range(1, 8):
    collected_df.append(sample_df(flist[i]),ignore_index = True)
    print("{0} is done.".format(flist[i]))

adj_SNS_2.xlsx is done.
adj_SNS_3.xlsx is done.
adj_SNS_4.xlsx is done.
adj_SNS_5.xlsx is done.
adj_SNS_6.xlsx is done.
adj_SNS_7.xlsx is done.
adj_SNS_8.xlsx is done.


In [16]:
collected_df.head()

,DOCID,SECTION,DATE,TITLE,CONTENT
38243,204053341,블로그,20180504,여드름압출 흉터 남기지않고 치료하기 1만원으로 로하셀에서 가능 야간진료까지 HSA,한방다이어트 여드름압출 흉터 남기지않고 치료하기 1만원으로 로하셀에서 가능 야간진료...
769,204116779,블로그,20180719,6월,엄마의 시간 6월 야옹이 2018 7 19 2353 URL 복사 이웃추가 본문 기...
51558,204098028,블로그,20180715,핀덴스킨베베 아기선크림과 바디워시 넘나 만족스럽네요,초보맘♡ 핀덴스킨베베 아기선크림과 바디워시 넘나 만족스럽네요 각설탕 2018 7 ...
104694,204037673,블로그,20180525,미세먼지 창문필터 실사용후기,생활꿀팁 미세먼지 창문필터 실사용후기 aa044 2018 5 25 2321 URL...
88010,204042772,블로그,20180505,시카 피니쉬 크림 극심한 민감함을 잠재우는 시카고 끝장크림,코스메틱메이크업 시카 피니쉬 크림 극심한 민감함을 잠재우는 시카고 끝장크림 소달 ...


In [17]:
collected_df.shape

(9824, 5)

In [19]:
collected_df.to_excel("Sampled dataframe.xlsx", index = False, engine = "xlsxwriter")

In [80]:
def doc_processing(i):
    startTime = time.time()
    print(startTime)
    data = pd.read_excel(i)
    processed_docs = data['CONTENT'].map(pre_processing)
    TrueEnd = time.time()- startTime
    print(TrueEnd)
    return processed_docs

In [81]:
i = 0
j = 0
def adjust(df):
    df = df.drop(["DOCID","TITLE"], axis = 1 ) 
    df["DATE"] = df["DATE"] // 10**6
    return df

def tokenizing(text):
    global i 
    global j
    i += 1 
    if i % 100 == 0 :
        print("{0}th iteration, time {1}".format(i, time.time()))
    #returns 형태소, exclude surplus tags.
    try:
        text = str(text)
        text = delet_re.sub('', text)
        tok = mecab.morphs(text)
        
    except UnicodeDecodeError:
        j += 1 
        print("{0}th text had been dumped..".format(j))
        return []
   
    return tok
    

def pre_processing(text) :
    result = []
        
    for token in tokenizing(text):
        if token not in stop_list :
            result.append(token)
        
    return result

In [82]:
delet_re = re.compile(r'[:/(){}\[\]\|@,;\\]+\w+|[:/(){}\[\]\|@,;\\]+|[0-9]+')

In [83]:
result = doc_processing("Sampled dataframe.xlsx")

1566793449.9999788
100th iteration, time 1566793451.8450017
200th iteration, time 1566793452.354002
300th iteration, time 1566793452.849977
400th iteration, time 1566793453.4029746
500th iteration, time 1566793454.0340042
600th iteration, time 1566793454.6809778
700th iteration, time 1566793455.2870018
800th iteration, time 1566793455.8300025
900th iteration, time 1566793456.4409778
1000th iteration, time 1566793456.9989789
1100th iteration, time 1566793457.6100018
1200th iteration, time 1566793458.1999745
1300th iteration, time 1566793458.7139747
1400th iteration, time 1566793459.263975
1500th iteration, time 1566793459.8099747
1600th iteration, time 1566793460.394977
1700th iteration, time 1566793461.0039773
1800th iteration, time 1566793461.5679746
1900th iteration, time 1566793462.1969745
2000th iteration, time 1566793462.7989771
2100th iteration, time 1566793463.481977
2200th iteration, time 1566793464.065977
2300th iteration, time 1566793464.8039777
2400th iteration, time 1566793

In [84]:
collected_list = []
for i in result :
    collected_list.extend(i)

In [85]:
collected_dict = collections.Counter(collected_list)

In [86]:
top_nlist = collected_dict.most_common(100)

In [87]:
print(top_nlist)

[('는', 177893), ('고', 102720), ('은', 86623), ('있', 77033), ('도', 68771), ('한', 63558), ('기', 54724), ('게', 47593), ('보', 34315), ('지', 32753), ('좋', 30649), ('수', 29557), ('되', 29440), ('해', 28364), ('주', 25340), ('면', 25001), ('었', 23359), ('어요', 20958), ('시', 20193), ('인', 20077), ('했', 19859), ('만', 19583), ('할', 19127), ('는데', 18203), ('적', 17371), ('없', 16748), ('먹', 16204), ('않', 16197), ('대', 16122), ('피부', 15542), ('같', 14955), ('곳', 14005), ('미세먼지', 13970), ('잘', 13962), ('안', 13275), ('니', 12866), ('성', 12839), ('ㅋㅋㅋ', 12747), ('입니다', 12565), ('더', 12528), ('사용', 12309), ('많', 12172), ('던', 11032), ('겠', 10834), ('거', 10751), ('합니다', 10663), ('내', 10642), ('았', 10622), ('ㅋㅋ', 10601), ('서', 10465), ('먼지', 10463), ('라', 10299), ('분', 10290), ('제품', 10064), ('살', 9976), ('역', 9429), ('추가', 9377), ('중', 9229), ('어서', 9171), ('네요', 9024), ('원', 8950), ('공기', 8803), ('이웃', 8584), ('너무', 8548), ('눈', 8501), ('미세', 8365), ('집', 8242), ('차', 8213), ('번역', 8098), ('해서', 8044), ('관리', 7